## Segmenting and Clustering Neighborhoods in Toronto #2

#### Import Libraries

In [ ]:
!conda install -c conda-forge geocoder --yes #installing beautiful soup
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

print('Libraries imported.')

Solving environment: / 

In [2]:
dfs = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M',header=0)
df = dfs[0] #only interestested in the first table

# Dropping all rows with an unassigned Borough
data1 = df[df.Borough != 'Not assigned']
data1.reset_index(inplace=True)
data1 = data1.drop(['index'], axis=1)

# Assigning Neighbourhood to Borough if not assigned
data1['Neighbourhood'] = np.where(data1.Neighbourhood == 'Not assigned', data1.Borough, data1.Neighbourhood)

data2 = data1.groupby(['Postcode', 'Borough'], sort=False)['Neighbourhood'].apply(lambda x: ', '.join(x))

data = pd.DataFrame(data2).reset_index(inplace=False)

# Getting the shape of the data
[maxRow,maxCol]=data.shape
maxRow,maxCol

(103, 3)

In [ ]:
import geocoder # import geocoder

# Setting parameters for start and end
start = data.index[1]       
end = data.index[maxRow-1]   

# initialize your variable to None
lat_lng_coords = None

for i in range(start,end+1,1):    # iterate all rows in the data

    # loop until you get the coordinates
    while(lat_lng_coords is None):
      g = geocoder.google(data.Borough[i] + ' ' + data.Postcode[i])
      lat_lng_coords = g.latlng

    data.latitude[i] = lat_lng_coords[0]
    data.longitude[i] = lat_lng_coords[1]  

In [ ]:
latlong_df = pd.read_csv("http://cocl.us/Geospatial_data")
latlong_df.columns = ['Postcode', 'Latitude', 'Longitude'] #renaming Postal Code to Post Code to match first dataframe
latlong_df

In [ ]:
finaldf = pd.merge(data, latlong_df)
finaldf